In [1]:
import os 
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt

In [2]:
import nibabel as nib
import pandas as pd
from glob import glob
import time
from tqdm import tqdm

In [3]:
import tensorflow as tf

2023-04-21 10:54:59.310008: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-21 10:54:59.680710: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-21 10:54:59.861714: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-21 10:55:01.376641: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [4]:
os.listdir(os.getcwd())

['np_X_data.pickle',
 'Data preprocessing_palmetto.ipynb',
 'Model build.ipynb',
 '2D data.ipynb',
 'Y_feature_rotate.pickle',
 'np_X_data_rotate.pickle',
 'Y_recognize.pickle',
 '.ipynb_checkpoints',
 '3d_facialrecognize.h5',
 'Feature loss Model.ipynb',
 'Y_feature.pickle',
 'Y_recognize_rotate.pickle',
 'DataPreprocessing_rotate.ipynb',
 'BSE_BET_DATA',
 '3d_brainLoss.h5',
 'new test data preprocess.ipynb']

In [5]:
os.chdir(os.getcwd() +'/BSE_BET_DATA')

In [6]:
os.getcwd()

'/home/cbasire/Data Mining/BSE_BET_DATA'

In [7]:
os.listdir(os.getcwd())#+ str('/files')

['labels.csv', 'DMimages', '.ipynb_checkpoints', 'files', 'mri.xlsx']

In [8]:
path1 = '/home/cbasire/Data Mining/BSE_BET_DATA/'
image_file_paths = glob(path1 + 'DMimages/*.nii')

In [9]:
image_file_paths[:5]

['/home/cbasire/Data Mining/BSE_BET_DATA/DMimages/IXI060-Guys-0709-MRA.nii',
 '/home/cbasire/Data Mining/BSE_BET_DATA/DMimages/IXI021-Guys-0703-MRA.nii',
 '/home/cbasire/Data Mining/BSE_BET_DATA/DMimages/IXI030-Guys-0708-MRA.nii',
 '/home/cbasire/Data Mining/BSE_BET_DATA/DMimages/IXI036-Guys-0736-MRA.nii',
 '/home/cbasire/Data Mining/BSE_BET_DATA/DMimages/IXI034-HH-1260-MRA.nii']

In [10]:
img = nib.load(image_file_paths[0]).get_fdata()
img.shape

(512, 512, 100)

In [11]:
# Read nifti file
def read_nifti_file(filepath):
    """Read and load volume"""
    # Read file
    scan = nib.load(filepath).get_fdata()
    return scan


# Normalize using min-max normalization
def normalize(volume):
    """Normalize the volume"""
    min = -1000
    max = 400
    volume[volume < min] = min
    volume[volume > max] = max
    volume = (volume - min) / (max - min)
    return volume.astype("float32")


# Resize image to standard size for all of them
def resize_volume(img):
    depth_factor = 1 / (img.shape[-1] / 64)
    width_factor = 1 / (img.shape[0] / 128)
    height_factor = 1 / (img.shape[1] / 128)
    # Rotate
    # img = ndimage.rotate(img, 90, reshape=False)
    # Resize across z-axis
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img


# All 3 functions above put in one
def process_scan(path):
    """Read and resize volume"""
    # Read scan
    volume = read_nifti_file(path)
    # Normalize
    volume = normalize(volume)
    # Resize width, height and depth
    volume = resize_volume(volume)
    return volume

In [12]:
data = pd.read_excel(path1+'mri.xlsx')

data.head()

,Filename,Recognizable-Facial-Feature,Brain-Feature-Loss
0,IXI002-Guys-0828-MRA.nii.gz,yes,No
1,IXI012-HH-1211-MRA.nii.gz,yes,No
2,IXI013-HH-1212-MRA.nii.gz,no,No
3,IXI015-HH-1258-MRA.nii.gz,yes,No
4,IXI016-Guys-0697-MRA.nii.gz,yes,No


In [13]:
df_recognize = data.iloc[: , :-1]
df_brainLoss = data.drop('Recognizable-Facial-Feature', axis=1)

print(df_recognize['Recognizable-Facial-Feature'].value_counts())
print(df_brainLoss['Brain-Feature-Loss'].value_counts())

Recognizable-Facial-Feature
Yes    61
No     59
yes     7
no      2
Name: count, dtype: int64
Brain-Feature-Loss
No    129
Name: count, dtype: int64


In [14]:
df_recognize['Recognizable-Facial-Feature'] = df_recognize['Recognizable-Facial-Feature'].replace({'Yes': 1, 'No': 0})

df_brainLoss['Brain-Feature-Loss'] = df_brainLoss['Brain-Feature-Loss'].replace({'Yes': 1, 'No': 0})

/local_scratch/pbs.533450.pbs02/ipykernel_617556/3161790267.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recognize['Recognizable-Facial-Feature'] = df_recognize['Recognizable-Facial-Feature'].replace({'Yes': 1, 'No': 0})


In [23]:
df_recognize_paths = []
for filename in df_recognize['Filename']:
    for path in image_file_paths:
        # Check if path is a Yes file or a No fi
        if path[48:] == filename[:-3]:
            df_recognize_paths.append(glob(path))

In [24]:
np_X_data = np.array([process_scan(path[0]) for path in tqdm(df_recognize_paths)])

100%|██████████| 62/62 [00:36<00:00,  1.69it/s]


In [25]:
Y_recognize_test = df_recognize.iloc[:,-1].to_numpy()

In [26]:
Y_feature_test = df_brainLoss.iloc[:,-1].to_numpy()

In [43]:
np_X_data.shape

TensorShape([62, 128, 128, 64, 1])

In [40]:
np_X_data = tf.squeeze(np_X_data,axis=3)

In [42]:
np_X_data = tf.expand_dims(np_X_data, axis=4)

In [44]:
with open('new_testdata.pickle', 'wb') as f:
    pickle.dump(np_X_data,f)
with open('Y_recognize_newtest.pickle', 'wb') as f:
    pickle.dump(Y_recognize_test,f)
with open('Y_feature_newtest.pickle', 'wb') as f:
    pickle.dump(Y_feature_test,f)